In [9]:
import merkletools
from web3 import Web3
from solcx import compile_files
import client.utils as utils
import client.ram as ram
import copy

In [ ]:
contract_id, compiled_sol = compile_files(["contracts/mismatch.sol:Mismatch"], output_values=['abi', 'bin']).popitem()

w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))

mismatch = w3.eth.contract(abi=compiled_sol['abi'], bytecode=compiled_sol['bin'])

tx_hash = mismatch.constructor().transact({'from': w3.eth.accounts[1]})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

Mismatch = w3.eth.contract(address=tx_receipt.contractAddress, abi=compiled_sol['abi'])


In [10]:
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
compiled_sol = compile_files(["contracts/mismatch.sol"], output_values=['abi', 'bin'])['contracts/mismatch.sol:Mismatch']
Mismatch = w3.eth.contract(address='0xBB8f430bb1229110Ac7677C995F39595a8f58018', abi=compiled_sol['abi'])

In [11]:
code = ram.RamCoder(10)

#fibonacci
code.load(1, 0)
code.load(1, 1)
code.load(2500, 3)
code.load(1, 4)
code.load(2, 5)
code.add(0, 1, 2)
code.copy_id(4, 0)
code.copy_id(5, 1)
code.sub(3, 4, 3)
code.jnz(3, 4)
code.end()

ram1 = ram.Ram(code.source)
ram1()

mt1 = utils.reduced_config_mt(ram1.rc_list)

In [12]:
corrupted_rc_list = copy.deepcopy(ram1.rc_list)

import random
lenght_rc = len(ram1.rc_list)
divergence = random.randint(0, lenght_rc - 1)
for i in range(lenght_rc):
    if i < divergence: continue
    corrupted_rc_list[i]["args"] = [1, 2, 3]

mt2 = utils.reduced_config_mt(corrupted_rc_list)

divergence

3375

In [ ]:
import pickle

rc_list = None

print('step 1')
with open('test.dictionary', 'rb') as config_dictionary_file:
    rc_list = pickle.load(config_dictionary_file)

print('step 2')
mt1 = utils.reduced_config_mt(rc_list)

print('step 3')
corrupted_rc_list = copy.deepcopy(rc_list)



In [ ]:
print('step 4')
for i in range(400000):
    corrupted_rc_list[12 + i]["args"] = [1, 2, 3]

print('step 5')
mt2 = utils.reduced_config_mt(corrupted_rc_list)

print('step 6')
mt2.get_leaf(11)

In [13]:
contractors = [w3.eth.accounts[1], w3.eth.accounts[2]]

tx_hash = Mismatch.functions.start_committed_bs(
    len(mt1.leaves), 
    '0x' + mt1.get_merkle_root(),
    '0x' + mt2.get_merkle_root(),
    contractors[0],
    contractors[1],
    '0x' + ram1.code_mt.get_merkle_root()).transact({'from': w3.eth.accounts[0]})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

key = '0x' + utils.calculate_key(
    len(mt1.leaves), 
    '0x' + mt1.get_merkle_root(),
    '0x' + mt2.get_merkle_root(),
    contractors[0],
    contractors[1],
    '0x' + ram1.code_mt.get_merkle_root())

tx_receipt.gasUsed

253427

In [ ]:
key

In [66]:
import asyncio
mt = [mt1, mt2]

def handle_event(event, conractor):
    middle = event['args']['middle']
    key = event['args']['key']
    Mismatch.functions.send_rc_proof(utils.translate_proof(mt[conractor-1], middle)).transact({'from': w3.eth.accounts[conractor]}, key)
    print(f'conractor = {conractor} middle = {middle}')

def handle_verify_committed_step(event, rc_list, conractor):
    low = event['args']['low']
    rc = rc_list[low]
    return Mismatch.functions.send_committed_step(
        PC = rc['PC'], 
        OpCode = int(rc['OpCode']), 
        args = rc['args'], 
        args_values = utils.sterilize_ints(rc['args_values']), 
        args_proof = rc['args_proofs'],
        store_mt_root = rc['store_mt_root'],
        nextOpCode = int(rc_list[low+1]['OpCode']),
        nextArgs = utils.sterilize_ints(rc_list[low+1]['args']),
        next_op_proof = rc['next_op_proof']
        ).transact({'from': w3.eth.accounts[conractor]})
    

async def log_loop_c1(event_filter, event_filter_verify, poll_interval):
    for x in range(8):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 1)
        for event in event_filter_verify.get_new_entries():
            pass
            #handle_verify_committed_step(event, ram1.rc_list, 1)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(event_filter, event_filter_verify, poll_interval):
    for x in range(8):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 2)
        await asyncio.sleep(poll_interval)

async def main():
    event_filter_c1 = Mismatch.events.request_proof_of_middle.createFilter(fromBlock=1 ,toBlock='latest')
    event_filter_c2 = Mismatch.events.request_proof_of_middle.createFilter(fromBlock=1 ,toBlock='latest')
    event_filter_verift_c1 = Mismatch.events.request_proof_of_committed_step.createFilter(fromBlock=1 ,toBlock='latest')
    event_filter_verift_c2 = Mismatch.events.request_proof_of_committed_step.createFilter(fromBlock=1 ,toBlock='latest')

    await asyncio.gather(
        log_loop_c1(event_filter_c1, event_filter_verift_c1, 1),
        log_loop_c2(event_filter_c2, event_filter_verift_c2, 1))


await main()


In [14]:
import asyncio
mt = [mt1, mt2]

c1_tx = []
c2_tx = []

def handle_event(event, conractor, tx_list):
    middle = event['args']['middle']
    #key = "0x" + event['args']['key'].hex()
    tx = Mismatch.functions.send_rc_proof(utils.translate_proof(mt[conractor], middle), key).transact({'from': contractors[conractor]})
    tx_list.append(tx)
    print(f'conractor = {conractor} middle = {middle}')

def handle_verify_committed_step(events, rc_list, conractor):
    event = None
    for e in events:
        if key == "0x" + e['args']['key'].hex():
            event = e
            break

    if event is None: return None

    low = event['args']['low']
    rc = rc_list[low]
    return Mismatch.functions.send_committed_step(
        PC = rc['PC'], 
        OpCode = int(rc['OpCode']), 
        args = rc['args'], 
        args_values = utils.sterilize_ints(rc['args_values']), 
        args_proof = rc['args_proofs'],
        store_mt_root = rc['store_mt_root'],
        nextOpCode = int(rc_list[low+1]['OpCode']),
        nextArgs = utils.sterilize_ints(rc_list[low+1]['args']),
        next_op_proof = rc['next_op_proof'],
        key = key
        ).transact({'from': contractors[conractor]})
    

async def log_loop_c1(poll_interval):
    old = []
    for x in range(20):
        dif = list(set(Mismatch.events.request_proof_of_middle.getLogs(fromBlock=1)) - set(old))
        old = list(set(Mismatch.events.request_proof_of_middle.getLogs(fromBlock=1)))
        for PairCreated in dif:
            if key == "0x" + PairCreated['args']['key'].hex():
                handle_event(PairCreated, 0, c1_tx)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(poll_interval):
    old = []
    for x in range(20):
        dif = list(set(Mismatch.events.request_proof_of_middle.getLogs(fromBlock=1)) - set(old))
        old = list(set(Mismatch.events.request_proof_of_middle.getLogs(fromBlock=1)))
        for PairCreated in dif:
            if key == "0x" + PairCreated['args']['key'].hex():
                handle_event(PairCreated, 1, c2_tx)
        await asyncio.sleep(poll_interval)

async def main():

    await asyncio.gather(
        log_loop_c1(1),
        log_loop_c2(1))


await main()

conractor = 0 middle = 7502
conractor = 1 middle = 7502
conractor = 0 middle = 3751
conractor = 1 middle = 3751
conractor = 0 middle = 1875
conractor = 1 middle = 1875
conractor = 0 middle = 2813
conractor = 1 middle = 2813
conractor = 0 middle = 3282
conractor = 1 middle = 3282
conractor = 0 middle = 3516
conractor = 1 middle = 3516
conractor = 0 middle = 3399
conractor = 1 middle = 3399
conractor = 0 middle = 3340
conractor = 1 middle = 3340
conractor = 0 middle = 3369
conractor = 1 middle = 3369
conractor = 0 middle = 3384
conractor = 1 middle = 3384
conractor = 0 middle = 3376
conractor = 1 middle = 3376
conractor = 0 middle = 3372
conractor = 1 middle = 3372
conractor = 0 middle = 3374
conractor = 1 middle = 3374
conractor = 0 middle = 3375
conractor = 1 middle = 3375


In [15]:

tx = handle_verify_committed_step(Mismatch.events.request_proof_of_committed_step.getLogs(fromBlock=1), ram1.rc_list, 1)
c1_tx.append(tx)


In [16]:
Mismatch.events.request_proof_of_middle.getLogs(fromBlock=1)
Mismatch.events.request_proof_of_committed_step.getLogs(fromBlock=1)
Mismatch.functions.get_winner(key).call()


'0x0b952Ffd52915C6D66e6cb740bd18B207B5Bf69e'

In [ ]:

total_cost_c1 = 0
for tx in c1_tx:
    receipt = w3.eth.waitForTransactionReceipt(tx)
    total_cost_c1 += receipt.gasUsed

total_cost_c2 = 0
for tx in c2_tx:
    receipt = w3.eth.waitForTransactionReceipt(tx)
    total_cost_c2 += receipt.gasUsed

print(total_cost_c1)
print(total_cost_c2)